In [1]:
import json

In [5]:
ground_truth_questions_file = '../../updated_pure_aggregate.json'

In [11]:
ground_truth_questions = []
with open(ground_truth_questions_file, 'r') as f:
    for line in f:
        temp = json.loads(line)
        ground_truth_questions.append(temp)

# Quick sanity check to make sure "special" feature values
# have been selected
if 'Sayville' not in ground_truth_questions[2]['question']:
    print "Check ground truth question file"